In [1]:
import cv2
import sys

In [2]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
from tensorflow import keras
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, BatchNormalization, Activation
from keras.initializers import GlorotUniform
from keras.models import Sequential, Model
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
from sklearn import preprocessing
from tensorflow.keras.preprocessing.image import img_to_array

%matplotlib inline

In [3]:
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
from sklearn.model_selection import train_test_split

In [4]:
img_folder="C:/Users/Amarir/Downloads/for research 1/Practical part/READY"
face_cascade=cv2.CascadeClassifier("C:/Users/Amarir/Downloads/haarcascade_frontalface_default.xml")
height=200
width=200,
image_size=(height,width,3)

In [5]:
import pathlib
img_folder = pathlib.Path(img_folder)
img_folder

WindowsPath('C:/Users/Amarir/Downloads/for research 1/Practical part/READY')

In [6]:
list(img_folder.glob('*/*.jpg'))[:5]


[WindowsPath('C:/Users/Amarir/Downloads/for research 1/Practical part/READY/S01/s01_01.jpg'),
 WindowsPath('C:/Users/Amarir/Downloads/for research 1/Practical part/READY/S01/s01_02.jpg'),
 WindowsPath('C:/Users/Amarir/Downloads/for research 1/Practical part/READY/S01/s01_03.jpg'),
 WindowsPath('C:/Users/Amarir/Downloads/for research 1/Practical part/READY/S01/s01_04.jpg'),
 WindowsPath('C:/Users/Amarir/Downloads/for research 1/Practical part/READY/S01/s01_05.jpg')]

In [7]:
def create_dataset(img_folder):
   
    img_data_array=[]
    class_name=[]
    classes=[]
   
    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):
       
            image_path= os.path.join(img_folder, dir1,  file)
            image= cv2.imread( image_path, cv2.COLOR_BGR2RGB)
            image=cv2.resize(image, (200, 200),interpolation = cv2.INTER_AREA)
            image=np.array(image)
            image = image.astype('float32')
            image /= 255 
            img_data_array.append(image)
            class_name.append(dir1)
            if dir1 not in classes:
                classes.append(dir1)

    return img_data_array, class_name,classes
# extract the image array and class name
img_data, class_name,classes =create_dataset("C:/Users/Amarir/Downloads/for research 1/Practical part/READY")

In [8]:
target_dict={k: v for v, k in enumerate(np.unique(class_name))}
target_val=  [target_dict[class_name[i]] for i in range(len(class_name))]

In [9]:
images_dict = {
    'S01': list(img_folder.glob('S01/*')),
    'S02': list(img_folder.glob('S02/*')),
    'S03': list(img_folder.glob('S03/*')),
    'S04': list(img_folder.glob('S04/*')),
    'S05': list(img_folder.glob('S05/*')),
    'S06': list(img_folder.glob('S06/*')),
    'S07': list(img_folder.glob('S07/*')),
    'S08': list(img_folder.glob('S08/*')),
    'S09': list(img_folder.glob('S09/*')),
    'S10': list(img_folder.glob('S10/*')),

    'S11': list(img_folder.glob('S11/*')),
    'S12': list(img_folder.glob('S12/*')),
    'S13': list(img_folder.glob('S13/*')),
    'S14': list(img_folder.glob('S14/*')),
    'S15': list(img_folder.glob('S15/*')),
    'S16': list(img_folder.glob('S16/*')),
    'S17': list(img_folder.glob('S17/*')),
    'S18': list(img_folder.glob('S18/*')),
    'S19': list(img_folder.glob('S19/*')),
    'S20': list(img_folder.glob('S20/*')),

    'S21': list(img_folder.glob('S21/*')),
    'S22': list(img_folder.glob('S22/*')),
    'S23': list(img_folder.glob('S23/*')),
    'S24': list(img_folder.glob('S24/*')),
    'S25': list(img_folder.glob('S25/*')),
    'S26': list(img_folder.glob('S26/*')),
    'S27': list(img_folder.glob('S27/*')),
    'S28': list(img_folder.glob('S28/*')),
    'S29': list(img_folder.glob('S29/*')),
    'S30': list(img_folder.glob('S30/*')),

    'S31': list(img_folder.glob('S31/*')),
    'S32': list(img_folder.glob('S32/*')),
    'S33': list(img_folder.glob('S33/*')),
    'S34': list(img_folder.glob('S34/*')),
    'S35': list(img_folder.glob('S35/*')),
    'S36': list(img_folder.glob('S36/*')),
    'S37': list(img_folder.glob('S37/*')),
    'S38': list(img_folder.glob('S38/*')),
    'S39': list(img_folder.glob('S39/*')),
    'S40': list(img_folder.glob('S40/*')),

    'S41': list(img_folder.glob('S41/*')),
    'S42': list(img_folder.glob('S42/*')),
    'S43': list(img_folder.glob('S43/*')),
    'S44': list(img_folder.glob('S44/*')),
    'S45': list(img_folder.glob('S45/*')),
    'S46': list(img_folder.glob('S46/*')),
    'S47': list(img_folder.glob('S47/*')),
    'S48': list(img_folder.glob('S48/*')),
    'S49': list(img_folder.glob('S49/*')),
    'S50': list(img_folder.glob('S50/*')),
    
    'safa': list(img_folder.glob('safa/*')),

}

In [10]:
X, y = [], []
for class_name, images in images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(200,200))
        X.append(resized_img)
        y.append(target_dict[class_name])



In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.2, random_state=0)

In [12]:
X = np.array(X)
y = np.array(y)

In [13]:
from keras.layers.preprocessing.image_preprocessing import RandomContrast
data_augmentation = keras.Sequential(
  [
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical",input_shape=(200,200,3)),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
    tf.keras.layers.experimental.preprocessing.RandomContrast(0.6),
  ]
)

In [14]:
model = keras.models.Sequential([
    data_augmentation,
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(200,200,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(51, activation='softmax')
])

In [15]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

In [16]:
checkpoint_path = 'C:/Users/Amarir/Downloads/for research 1/Practical part/new weights/Alexnet_GT20_80my_face.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [17]:
model.load_weights(checkpoint_path)

In [18]:
def draw_ped(img,label, x0, y0, xt, yt, color=(255,127,0), text_color=(255,255,255)):

    (w, h), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    cv2.rectangle(img,(x0, y0 + baseline),  (max(xt, x0 + w), yt),  color, 2)
    cv2.rectangle(img,(x0, y0 - h),  (x0 + w, y0 + baseline), color, -1)  
    cv2.putText(img, label, (x0, y0),cv2.FONT_HERSHEY_SIMPLEX,0.5,text_color,1,cv2.LINE_AA) 
    return img

In [ ]:
classes=np.flip(classes,0)
classes=np.sort(classes)

In [24]:
label_text=""
cap = cv2.VideoCapture(0)
while cap.isOpened() :
    ret, frame = cap.read()
    if ret:
        color = cv2.cvtColor(frame, cv2.COLOR_RGBA2RGB)
        faces = face_cascade.detectMultiScale(color, 1.1, 5)
        for (x, y, w, h) in faces:
            face_img = gray[y:y+h, x:x+w]
            face_img = cv2.resize(face_img,(200,200))
            face_img = img_to_array(face_img)
            face_img = np.expand_dims(face_img, axis=0)
            
            confidence = model.predict(face_img)[0]
            idx = np.argmax(confidence)
            label = classes[idx]
            label_text = "%s (%.2f %%)" % (label, confidence[idx]*100)
            
            frame = draw_ped(frame,label_text, x, y, x + w, y + h, color=(0,255,255), text_color=(50,50,50))
       
        cv2.imshow('Detect Face', frame)
    else :
        break
    if cv2.waitKey(10) == ord('q'):
        break
        
cv2.destroyAllWindows()
cap.release()

In [20]:
cv2.destroyAllWindows()
cap.release()